Sample 3 ORES scoring
===



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

In [2]:
import os
from tqdm import tqdm
import bz2
import gzip
import json
import re
import hashlib
from datetime import datetime
import nltk
import scipy.stats
import para
from itertools import groupby
from collections import Counter

In [3]:
git_root_dir = !git rev-parse --show-toplevel
git_root_dir = git_root_dir[0]
git_root_dir

'/export/scratch2/levon003/repos/wiki-ores-feedback'

In [4]:
raw_data_dir = "/export/scratch2/wiki_data"
derived_data_dir = os.path.join(git_root_dir, "data", "derived")
raw_data_dir, derived_data_dir

('/export/scratch2/wiki_data',
 '/export/scratch2/levon003/repos/wiki-ores-feedback/data/derived')

In [5]:
stub_history_dir = os.path.join(derived_data_dir, 'stub-history-all-revisions')
stub_history_dir

'/export/scratch2/levon003/repos/wiki-ores-feedback/data/derived/stub-history-all-revisions'

In [6]:
working_dir = os.path.join(derived_data_dir, 'revision_sample')
os.makedirs(working_dir, exist_ok=True)
working_dir

'/export/scratch2/levon003/repos/wiki-ores-feedback/data/derived/revision_sample'

In [7]:
start_date = datetime.fromisoformat('2014-04-01')
start_timestamp = int(start_date.timestamp())
end_date = datetime.fromisoformat('2020-01-01')
end_timestamp = int(end_date.timestamp())
start_timestamp, end_timestamp

(1396328400, 1577858400)

### Load sample 3


In [8]:
# read in the sample dataframe
revision_sample_dir = os.path.join(derived_data_dir, 'revision_sample')
sample3_filepath = os.path.join(revision_sample_dir, 'sample3_all.pkl')
rev_df = pd.read_pickle(sample3_filepath)
len(rev_df)

33964442

In [20]:
# read in the ORES scores
# as sample3 is a superset of sample2 and overlaps with sample1, we also load those scores
revision_sample_dir = os.path.join(derived_data_dir, 'revision_sample')
ores_dfs = []
for fname in ['sample1_ores_scores.csv', 'sample2_ores_scores.csv', 'sample3_ores_scores.csv']:
    sample_filepath = os.path.join(revision_sample_dir, fname)
    ores_dfs.append(pd.read_csv(sample_filepath, header=None, names=['rev_id', 'damaging_prob', 'damaging_pred', 'goodfaith_prob', 'goodfaith_pred']))
ores_df = pd.concat(ores_dfs, axis=0)
# remove invalid scores
ores_df = ores_df[(ores_df.damaging_prob != -1)&(ores_df.goodfaith_prob != -1)]
# remove duplicates
ores_df = ores_df.drop_duplicates(subset='rev_id')
len(ores_df)

5932469

In [21]:
ores_df.head()

,rev_id,damaging_prob,damaging_pred,goodfaith_prob,goodfaith_pred
0,846835190,0.014069,False,0.988118,True
1,923997118,0.597491,True,0.550084,True
2,867368228,0.304521,False,0.827948,True
3,888337768,0.010122,False,0.995091,True
4,917010365,0.013616,False,0.989612,True


In [22]:
revs_with_ores_scores = set(ores_df.rev_id)
rev_df_subset = rev_df[~rev_df.rev_id.isin(revs_with_ores_scores)]
len(rev_df_subset)

28538373

In [23]:
rev_df_subset.sort_values(by='rev_timestamp').head()

,page_id,rev_id,rev_timestamp,is_revert_target,is_reverted,is_reverting,is_sample_eligible,prev_rev_id,next_rev_id,prev_rev_timestamp,next_rev_timestamp,reverted_rev_ids,reverting_rev_id,reverting_rev_timestamp
4034321,102136,818010205,1514786428,1,0,1,True,818010162,818320666,1514786411,1514951633,[818010162],-1,-1
47428238,33508060,818010223,1514786440,0,0,1,True,818010212,825091028,1514786435,1518373861,[818010212],-1,-1
65996032,56165762,818010239,1514786446,0,0,1,True,818010196,818010304,1514786426,1514786478,[818010196],-1,-1
50856007,38540988,818010257,1514786454,0,0,0,True,818010197,821578386,1514786426,1516550277,[],-1,-1
66027226,56182501,818034637,1514786454,0,0,0,True,818012784,818034638,1514787637,1514798793,[],-1,-1
